In [ ]:
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/feature/fix_ptuning_bug/requirements.txt -O requirements.txt


--2022-05-06 10:06:26--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/feature/fix_ptuning_bug/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 610 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     610  --.-KB/s    in 0s      

2022-05-06 10:06:26 (34.6 MB/s) - ‘requirements.txt’ saved [610/610]



In [ ]:
!pip  install -r requirements.txt

     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.8 MB 65.4 MB/s 
     |████████████████████████████████| 1.0 MB 78.2 MB/s 
     |████████████████████████████████| 47.3 MB 1.1 MB/s 
     |████████████████████████████████| 252 kB 74.2 MB/s 
     |████████████████████████████████| 2.9 MB 58.2 MB/s 
     |████████████████████████████████| 19.4 MB 38.0 MB/s 
     |████████████████████████████████| 17.3 MB 33.6 MB/s 
     |████████████████████████████████| 880 kB 78.4 MB/s 
     |████████████████████████████████| 2.9 MB 70.1 MB/s 
     |████████████████████████████████| 11.3 MB 70.7 MB/s 
     |████████████████████████████████| 11.3 MB 77.6 MB/s 
     |████████████████████████████████| 11.3 MB 84.7 MB/s 
     |████████████████████████████████| 11.5 MB 27.0 MB/s 
     |████████████████████████████████| 10.8 MB 76.5 MB/s 
     |████████████████████████████████| 9.9 MB 76.3 MB/s 
     |████

In [ ]:
import transformers
transformers.__version__

'4.2.2'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
import random

from transformers import AutoTokenizer, TFGPT2Model , TFGPT2LMHeadModel , TFAutoModelForSequenceClassification ,TFAutoModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time

In [ ]:
print(np.__version__)
print(tf.__version__)

1.16.2
2.7.1


In [ ]:
SEED = 28
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")


train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')


train_data.drop_duplicates(subset = ['document'], inplace=True) 
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = train_data['document'].str.replace('^ +', "") 
train_data['document'].replace('', np.nan, inplace=True) 
train_data = train_data.dropna(how='any') 

train_data.rename(columns = {'document' : 'text'}, inplace = True)
train_data.reset_index(drop=True)
train_data.drop(columns=['id'] ,  inplace = True)

test_data.drop_duplicates(subset = ['document'], inplace=True) 
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
test_data['document'] = test_data['document'].str.replace('^ +', "") 
test_data['document'].replace('', np.nan, inplace=True) 
test_data = test_data.dropna(how='any') 

test_data.rename(columns = {'document' : 'text'}, inplace = True)
test_data.reset_index(drop=True)
test_data.drop(columns=['id'] , inplace = True)

In [ ]:
steam_data.head()

,0,노래가 너무 적음
0,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
1,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
2,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
3,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
4,1,역시 재미있네요 전작에서 할수 없었던 자유로운 덱 빌딩도 좋네요^^


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>' , sep_token = '<sep>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
print(tokenizer.encode("부정"))

[11775]


In [ ]:
print(tokenizer.sep_token)
print(tokenizer.encode("<sep>"))

<sep>
[51201]


In [ ]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [ ]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [ ]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [ ]:
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

보는
내
내
그대로
들어
맞
는
예측
카
리스
마
없는
악
역


In [ ]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['▁전', '율을', '▁일으키는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']


In [ ]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]


In [ ]:
for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
  print(tokenizer.decode(elem))

전
율을
일으키는
영화
.
다시
보고
싶
은
영화


In [ ]:
for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))


ha
pp
y
b
ir
th
d
ay
~
!


In [ ]:
print(tokenizer.decode(3))

<pad>


In [ ]:
max_seq_len = 128

In [ ]:
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",truncation=True, max_length=max_seq_len, padding="max_length")
print(encoded_result)
print('길이 :', len(encoded_result))

[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128


In [ ]:

max_seq_len = 128
import random


train_data_sents = []
train_attn_mask = []
train_data_labels = []

for train_sent, train_label in tqdm(train_data[['text', 'label']].values):
    train_text_label = '긍정' if train_label == 1 else '부정'

    
    tokens1 = [tokenizer.bos_token] + tokenizer.tokenize(train_sent) 
    train_tokenized_text = tokenizer.convert_tokens_to_ids(tokens1)
    tokens = pad_sequences([train_tokenized_text], maxlen=max_seq_len+1, value=tokenizer.pad_token_id, padding='post')[0]
    
    
    tokens = np.insert(tokens, 0, 0) #add first promt
    tokens = np.insert(tokens, len(tokens), 1) #add last promt
    
   
    train_attn_mask.append([1 if t != 3 else 0 for t in tokens])
    train_data_sents.append(tokens)

    label = tokenizer.encode("긍정") if train_label == 1 else  tokenizer.encode("부정")
    train_data_labels.append(label)


train_attn_mask = np.array(train_attn_mask, dtype=np.int64)
train_data_sents = np.array(train_data_sents, dtype=np.int64)
train_data_labels = np.array(train_data_labels, dtype=np.int64)

100%|██████████| 145393/145393 [01:15<00:00, 1926.77it/s]


In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2LMHeadModel.from_pretrained(model_name, from_pt=True)
        self.gpt.trainable = False

        self.prompt_embedding_size = self.gpt.config.hidden_size
        self.prompt_emgedding = tf.keras.layers.Embedding(2, self.prompt_embedding_size, name='prompt_embedding')
        
        self.bilstm = tf.keras.Sequential(name='prompt_bilstm')
        self.bilstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.prompt_embedding_size, return_sequences=True)))
        self.bilstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.prompt_embedding_size, return_sequences=True)))

        self.mlp = tf.keras.Sequential(name='prompt_mlp')
        self.mlp.add(tf.keras.layers.Dense(self.prompt_embedding_size))
        self.mlp.add(tf.keras.layers.ReLU())
        self.mlp.add(tf.keras.layers.Dense(self.prompt_embedding_size))



        self.bigru1 = tf.keras.Sequential(name='prompt_bigru1')
        self.bigru1.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(self.prompt_embedding_size, return_sequences=True)))

        self.mlp1 = tf.keras.Sequential(name='prompt_mlp1')
        self.mlp1.add(tf.keras.layers.Dense(self.prompt_embedding_size))
        self.mlp1.add(tf.keras.layers.ReLU())
        self.mlp1.add(tf.keras.layers.Dense(self.prompt_embedding_size))
        

    def generate_prompt_input(self, inputs_ids):
        
        p1 = self.gpt.transformer.wte(inputs_ids[ :, 1:-1])
        p2 = self.bilstm1(p1)
        inputs_embeds = self.mlp1(p2)

        prompt_indexs = tf.concat([inputs_ids[:, 0:1],inputs_ids[:, -1:]], axis=-1)
        
        prompt_embeds = self.prompt_emgedding(prompt_indexs)
        
        prompt_embeds = self.bilstm(prompt_embeds)
        
        prompt_embeds = self.mlp(prompt_embeds)
       
        prompt_updated_inputs = tf.concat([prompt_embeds[:, 0:1, :], inputs_embeds ,  prompt_embeds[:, 1:, :]], axis=1)
      
        return prompt_updated_inputs
        
    def call(self, inputs):
        input_ids = inputs[0]
        attention_mask = inputs[1] if len(inputs) > 1 else None
        inputs_embeds = self.generate_prompt_input(input_ids)
        
        outputs = self.gpt({'inputs_ids': None, 'inputs_embeds': inputs_embeds, 'attention_mask': attention_mask})

        output = outputs.logits[:, -1, :]
        return output

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping , ModelCheckpoint

In [ ]:
def lr_schedule(epoch):
    
    if epoch < 20:
        return 1e-5 
    if epoch < 30:
        return 1e-5 * 0.1
    if epoch < 35:
        return 1e-5 * 0.01
    return 1e-5 * 0.001

In [ ]:
with strategy.scope():
    model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    lr_sh = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
for layer in model.layers[:]:
    print(layer.trainable)

False
True
True


In [ ]:
model_name = "tf2_gpt2_ptuning_naver_movie"
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = "./data_out/KOR"

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=3)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)


history = model.fit( (train_data_sents, train_attn_mask), train_data_labels, 
                     epochs=40, batch_size=256, validation_split=0.1 , callbacks=[earlystop_callback, cp_callback ,lr_sh])

In [ ]:
trainableParams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
totalParams = trainableParams + nonTrainableParams
 
print(trainableParams)
print(nonTrainableParams)
print(totalParams)

In [ ]:
model.load_weights('/content/data_out/KOR/tf2_gpt2_ptuning_naver_movie/weights.h5')

In [ ]:
# test_data = test_data[:50] # for test

test_data_sents = []
test_data_labels = []
test_attn_mask = []

pred_tokens = []


for test_sent, test_label in test_data[['text', 'label']].values:

    tokens1 = [tokenizer.bos_token] + tokenizer.tokenize(test_sent) 
    train_tokenized_text = tokenizer.convert_tokens_to_ids(tokens1)
    
    tokens = pad_sequences([train_tokenized_text], maxlen=max_seq_len+1, value=tokenizer.pad_token_id, padding='post')[0]
    
    tokens = np.insert(tokens, 0, 0) #add first promt
    tokens = np.insert(tokens, len(tokens), 1) #add last promt
    test_data_sents.append(tokens)
    
    mask = [1 if t != 3 else 0 for t in tokens]
    test_attn_mask.append(mask)

    label = tokenizer.encode("긍정") if test_label == 1 else  tokenizer.encode("부정")
    test_data_labels.append(label)
    
test_attn_mask = np.array(test_attn_mask, dtype=np.int64)
test_data_sents = np.array(test_data_sents, dtype=np.int64)
test_data_labels = np.array(test_data_labels, dtype=np.int64)

results = model.evaluate((test_data_sents, test_attn_mask), test_data_labels, batch_size=1024)
print("test loss, test acc: ", results)